In [16]:
from langchain_community.document_loaders import TextLoader # Take raw text of descriptions and convert into format langchain can work with

from langchain_text_splitters import CharacterTextSplitter # Split whole document containing descriptions into meaningful chunks
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma # Vector database (others can be used)

In [17]:
from dotenv import load_dotenv
load_dotenv()

True

In [18]:
import pandas as pd

books = pd.read_csv('books_cleaned.csv')

In [19]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,missing_description,age_of_book,words_count,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,0,21.0,199,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web : A Novel,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,0,25.0,205,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,0,32.0,57,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,0,23.0,45,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,0,23.0,75,"9780006280934 ""In The Problem of Pain, C.S. Le..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,0,22.0,288,9788172235222 On A Train Journey Home To North...
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,0,23.0,63,9788173031014 This book tells the tale of a ma...
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,0,22.0,117,9788179921623 Wisdom to Create a Life of Passi...
5195,9788185300535,8185300534,I Am that : Talks with Sri Nisargadatta Maharaj,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,0,26.0,174,9788185300535 This collection of the timeless ...


In [20]:
books['tagged_description'].to_csv("tagged_description.txt", index=False, header=False)

In [21]:
raw_documents = TextLoader("tagged_description.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0, separator="\n")
documents = text_splitter.split_documents(raw_documents)

Created a chunk of size 1170, which is longer than the specified 1000
Created a chunk of size 1216, which is longer than the specified 1000
Created a chunk of size 1090, which is longer than the specified 1000
Created a chunk of size 1191, which is longer than the specified 1000
Created a chunk of size 1267, which is longer than the specified 1000
Created a chunk of size 2010, which is longer than the specified 1000
Created a chunk of size 1225, which is longer than the specified 1000
Created a chunk of size 1186, which is longer than the specified 1000
Created a chunk of size 1216, which is longer than the specified 1000
Created a chunk of size 1193, which is longer than the specified 1000
Created a chunk of size 1059, which is longer than the specified 1000
Created a chunk of size 1272, which is longer than the specified 1000
Created a chunk of size 1637, which is longer than the specified 1000
Created a chunk of size 1132, which is longer than the specified 1000
Created a chunk of s

In [22]:
documents[0]

Document(metadata={'source': 'tagged_description.txt'}, page_content='"9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, G

In [23]:
db_books = Chroma.from_documents(documents, embedding=OpenAIEmbeddings())

In [24]:
query = "A book about the maths of ai"
docs = db_books.similarity_search(query, k=5)

In [25]:
docs

[Document(id='416f105e-0686-4beb-b08c-c3faf8339bc8', metadata={'source': 'tagged_description.txt'}, page_content='"9780691050843 Presents a series of lectures delivered in 1994 by Hawking and Penrose, renowned professors at Cambridge and Oxford, respectively, on the general topic of how mathematical physics might best represent the realities of the universe."'),
 Document(id='f6bc60e5-eb43-47f4-8c76-a8077c1eeb64', metadata={'source': 'tagged_description.txt'}, page_content='"9780762419227 Looks at landmark mathematical discoveries over the past 2500 years by such mathematicians as Euclid, Isaac Newton, Pierre Simon de Laplace, Gerog Cantor, and Alan Turing."\n9780762427321 Hawking\'s revolutionary look at the scientific discoveries that changed people\'s perceptions of the world now comes complete with color photographs and illustrations depicting theoretical models of the planets and their orbits.\n"9780763619312 When a third grade classmate gets her picture in the paper for winning a

In [36]:
# Very simple top_k recommendation

def retrieve_semantic_recommendations(query, top_k=10):
    recs = db_books.similarity_search(query, k=top_k)

    books_list = []

    for i in range(0, len(recs)):
        isbn = int(recs[i].page_content.strip('"').split()[0])
        books_list.append(isbn)

    return books[books["isbn13"].isin(books_list)]

In [37]:
retrieve_semantic_recommendations(query)

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,missing_description,age_of_book,words_count,tagged_description
1851,9780385493628,0385493622,Fermat's Enigma : The Epic Quest to Solve the ...,Simon Singh,Mathematics,http://books.google.com/books/content?id=wkJ6Q...,"The story of Fermat's Last Theorem, devised by...",1997.0,4.24,315.0,18495.0,0,28.0,32,9780385493628 The story of Fermat's Last Theor...
1922,9780393324464,039332446X,The Future of Spacetime,Stephen W. Hawking;Kip S. Thorne;Igor D. Novik...,Science,http://books.google.com/books/content?id=LlVcB...,Presents essays that explore the deepest myste...,2003.0,3.93,224.0,241.0,0,22.0,32,9780393324464 Presents essays that explore the...
2405,9780451450647,0451450647,Robot Visions,Isaac Asimov,Fiction,http://books.google.com/books/content?id=6lYJA...,"Spanning Asimov's fifty-year career, this coll...",1991.0,4.22,496.0,7425.0,0,34.0,36,9780451450647 Spanning Asimov's fifty-year car...
2700,9780553345841,0553345842,The Mind's I : Fantasies and Reflections on Se...,Douglas R. Hofstadter;Daniel Clement Dennett,Philosophy,http://books.google.com/books/content?id=hz22Q...,"Brilliant, shattering, mind-jolting, ""The Mind...",1982.0,4.14,512.0,4715.0,0,43.0,88,"9780553345841 Brilliant, shattering, mind-jolt..."
2873,9780593040591,0593040597,The Illustrated A Brief History of Time,Stephen W. Hawking,Cosmology,http://books.google.com/books/content?id=pe7TO...,"This is Stephen Hawking's updated, expanded an...",1996.0,4.16,259.0,307.0,0,29.0,91,9780593040591 This is Stephen Hawking's update...
3235,9780691050843,0691050848,The Nature of Space and Time,Stephen Hawking;Roger Penrose,Science,http://books.google.com/books/content?id=LstaQ...,Presents a series of lectures delivered in 199...,2000.0,4.09,152.0,945.0,0,25.0,35,9780691050843 Presents a series of lectures de...
3474,9780747569138,0747569134,More Sideways Arithmetic from Wayside School,Louis Sachar,Arithmetic,http://books.google.com/books/content?id=EMj7H...,Welcome back to maths class at Wayside School....,2004.0,3.92,112.0,839.0,0,21.0,114,9780747569138 Welcome back to maths class at W...
3578,9780762419227,0762419229,God Created The Integers,Stephen Hawking,Mathematics,http://books.google.com/books/content?id=3zdFS...,Looks at landmark mathematical discoveries ove...,2005.0,4.06,1160.0,1650.0,0,20.0,26,9780762419227 Looks at landmark mathematical d...
4318,9781400032716,1400032717,The curious incident of the dog in the night-t...,Mark Haddon,Fiction,NaN,Despite his overwhelming fear of interacting w...,2003.0,3.87,226.0,991627.0,0,22.0,30,9781400032716 Despite his overwhelming fear of...
4340,9781400040308,1400040302,A Madman Dreams of Turing Machines,Janna Levin,Fiction,http://books.google.com/books/content?id=EwQi6...,"In a saga of genius, madness, and seminal scie...",2006.0,3.68,230.0,1098.0,0,19.0,42,"9781400040308 In a saga of genius, madness, an..."
